В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [6]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook

In [7]:
X = pd.read_csv('weather.csv')

In [8]:
X

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,145454,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,...,59.0,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No,No
142189,145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
142190,145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
142191,145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No


In [9]:
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})

In [10]:
del X['RainTomorrow']

In [11]:
X.T

,0,1,2,3,4,5,6,7,8,9,...,142183,142184,142185,142186,142187,142188,142189,142190,142191,142192
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,145449,145450,145451,145452,145453,145454,145455,145456,145457,145458
Date,2008-12-01,2008-12-02,2008-12-03,2008-12-04,2008-12-05,2008-12-06,2008-12-07,2008-12-08,2008-12-09,2008-12-10,...,2017-06-15,2017-06-16,2017-06-17,2017-06-18,2017-06-19,2017-06-20,2017-06-21,2017-06-22,2017-06-23,2017-06-24
Location,Albury,Albury,Albury,Albury,Albury,Albury,Albury,Albury,Albury,Albury,...,Uluru,Uluru,Uluru,Uluru,Uluru,Uluru,Uluru,Uluru,Uluru,Uluru
MinTemp,13.4,7.4,12.9,9.2,17.5,14.6,14.3,7.7,9.7,13.1,...,2.6,5.2,6.4,8.0,7.4,3.5,2.8,3.6,5.4,7.8
MaxTemp,22.9,25.1,25.7,28.0,32.3,29.7,25.0,26.7,31.9,30.1,...,22.5,24.3,23.4,20.7,20.6,21.8,23.4,25.3,26.9,27.0
Rainfall,0.6,0.0,0.0,0.0,1.0,0.2,0.0,0.0,0.0,1.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Evaporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sunshine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WindGustDir,W,WNW,WSW,NE,W,WNW,W,W,NNW,W,...,S,E,ESE,ESE,E,E,E,NNW,N,SE
WindGustSpeed,44.0,44.0,46.0,24.0,41.0,56.0,50.0,35.0,80.0,28.0,...,19.0,24.0,31.0,41.0,35.0,31.0,31.0,22.0,37.0,28.0


### Предобработка признаков

Удалим колонку "Unnamed: 0" так как она не несет в себе полезной информации.

In [12]:
del X["Unnamed: 0"]

Расположение безусловно важно при предсказании погоды, но с учетом того что данные отсортированы по названиям городов, и для обучения выбираются первые 75% можно предположить, что данный признак будет бесполезен при обучении модели. Поэтому удалим данный признак из датасета.

In [13]:
del X["Location"]

Заменим 'Yes' и 'No' в колонке RainToday на 1 и 0 соответственно.

In [14]:
X["RainToday"] = X.RainToday.replace({'No':0, 'Yes': 1})
X["RainToday"].value_counts()

0.0    109332
1.0     31455
Name: RainToday, dtype: int64

Заполним пропуски в численных признаках. В колонке RainToday пропуски заполним нулями, а в остольных - средним значением.

In [15]:
for i in X.describe().columns:
    X[i].fillna(X[i].mean(), inplace=True)

X["RainToday"].fillna(0, inplace=True)
X.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
count,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000,142193.000000
mean,12.186400,23.226784,2.349974,5.469824,7.624853,39.984292,14.001988,18.637576,68.843810,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.987509,21.687235,0.223423
std,6.388924,7.109554,8.423217,3.168114,2.734927,13.138385,8.851082,8.721551,18.932077,20.532065,6.746248,6.681788,2.278080,2.104709,6.472166,6.870771,0.414476
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,0.000000
25%,7.600000,17.900000,0.000000,4.000000,7.624853,31.000000,7.000000,13.000000,57.000000,37.000000,1013.500000,1011.000000,3.000000,4.000000,12.300000,16.700000,0.000000
50%,12.000000,22.700000,0.000000,5.469824,7.624853,39.000000,13.000000,18.637576,70.000000,51.482606,1017.653758,1015.258204,4.437189,4.503167,16.800000,21.300000,0.000000
75%,16.800000,28.200000,0.800000,5.469824,8.700000,46.000000,19.000000,24.000000,83.000000,65.000000,1021.800000,1019.400000,6.000000,6.000000,21.500000,26.300000,0.000000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,1.000000


В колонке "Date" оставим информацию только о месяце.

In [16]:
X["Date"] = X["Date"].apply(lambda x: int(x[5:7]))

Применим метод OneHotEncoder к котегориальным признакам

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

X["WindGustDir"] = LabelEncoder().fit_transform(X["WindGustDir"])
X["WindDir9am"] = LabelEncoder().fit_transform(X["WindDir9am"])
X["WindDir3pm"] = LabelEncoder().fit_transform(X["WindDir3pm"])
X["Date"] = LabelEncoder().fit_transform(X["Date"])

ohe = OneHotEncoder(sparse=False)
new_ohe_wind_gus_dir = ohe.fit_transform(X[["WindGustDir"]])
new_ohe_wind_dir_9am = ohe.fit_transform(X[["WindDir9am"]])
new_ohe_wind_dir_3am = ohe.fit_transform(X[["WindDir3pm"]])
new_ohe_date = ohe.fit_transform(X[["Date"]])

X_ohe = X.copy()

del X_ohe["WindGustDir"], X_ohe["WindDir9am"], X_ohe["WindDir3pm"], X_ohe["Date"]
X_ohe = pd.DataFrame(np.hstack((X_ohe, new_ohe_wind_gus_dir, new_ohe_wind_dir_9am, new_ohe_wind_dir_3am, new_ohe_date)))

Применим MinMaxScaler

In [18]:
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
X_ohe = mm_scaler.fit_transform(X_ohe)

Создадим датасеты без использования OneHotEncoder для метода ближайших соседей и байесовского классификатора. Также применим к ним MinMaxScaler и StandartScaler.

In [19]:
X_mm = mm_scaler.fit_transform(X)

s_scaler = preprocessing.StandardScaler()
X_ss = s_scaler.fit_transform(X)

### Логистическая регрессия

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.25, shuffle=False)

model = LogisticRegression()

In [107]:
%%timeit -n1
model.fit(X_train, y_train)

1 loop, best of 5: 2.81 s per loop


In [108]:
%%timeit -n1
y_predict = model.predict(X_test)

1 loop, best of 5: 3.79 ms per loop


In [23]:
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91     27882
           1       0.75      0.47      0.58      7667

    accuracy                           0.85     35549
   macro avg       0.81      0.71      0.74     35549
weighted avg       0.84      0.85      0.84     35549



### Метод ближайших соседей

Рассмотрим как разные методы масштабирования данных влияют на качество метода ближайших соседей.

In [110]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1)

In [111]:
%%timeit -n1
model.fit(X_train, y_train)

1 loop, best of 5: 12.1 ms per loop


In [112]:
%%timeit -n1
y_predict = model.predict(X_test)

1 loop, best of 5: 48.4 s per loop


In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

print(" " * 20 + "Without scaler:")
for i in [2, 5, 10, 15, 30]:
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    print("n_neighbors:", i)
    print(classification_report(y_test, model.predict(X_test)))

                    Without scaler:
n_neighbors: 2
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     27882
           1       0.68      0.31      0.43      7667

    accuracy                           0.82     35549
   macro avg       0.76      0.63      0.66     35549
weighted avg       0.80      0.82      0.79     35549

n_neighbors: 5
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     27882
           1       0.66      0.45      0.54      7667

    accuracy                           0.83     35549
   macro avg       0.76      0.69      0.72     35549
weighted avg       0.82      0.83      0.82     35549

n_neighbors: 10
              precision    recall  f1-score   support

           0       0.85      0.96      0.90     27882
           1       0.75      0.39      0.52      7667

    accuracy                           0.84     35549
   macro avg       0.80      0.68      0.71   

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X_mm, y, test_size=0.25, shuffle=False)

print(" " * 20 + "MinMaxScaler:")
for i in [1, 2, 5, 10, 15, 30]:
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    print("n_neighbors:", i)
    print(classification_report(y_test, model.predict(X_test)))

                    MinMaxScaler:
n_neighbors: 1
              precision    recall  f1-score   support

           0       0.86      0.88      0.87     27882
           1       0.51      0.46      0.48      7667

    accuracy                           0.79     35549
   macro avg       0.68      0.67      0.67     35549
weighted avg       0.78      0.79      0.78     35549

n_neighbors: 2
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     27882
           1       0.68      0.28      0.39      7667

    accuracy                           0.82     35549
   macro avg       0.75      0.62      0.64     35549
weighted avg       0.80      0.82      0.78     35549

n_neighbors: 5
              precision    recall  f1-score   support

           0       0.86      0.94      0.90     27882
           1       0.66      0.43      0.52      7667

    accuracy                           0.83     35549
   macro avg       0.76      0.68      0.71     3

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.25, shuffle=False)

print(" " * 20 + "StandartScaler:")
for i in [2, 5, 10, 15, 30]:
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    print("n_neighbors:", i)
    print(classification_report(y_test, model.predict(X_test)))

                    StandartScaler:
n_neighbors: 2
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     27882
           1       0.66      0.28      0.39      7667

    accuracy                           0.81     35549
   macro avg       0.74      0.62      0.64     35549
weighted avg       0.79      0.81      0.78     35549

n_neighbors: 5
              precision    recall  f1-score   support

           0       0.86      0.93      0.89     27882
           1       0.63      0.43      0.51      7667

    accuracy                           0.82     35549
   macro avg       0.75      0.68      0.70     35549
weighted avg       0.81      0.82      0.81     35549

n_neighbors: 10
              precision    recall  f1-score   support

           0       0.85      0.96      0.90     27882
           1       0.73      0.37      0.49      7667

    accuracy                           0.83     35549
   macro avg       0.79      0.66      0.69   

###  Байесовский классификатор

In [116]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

In [117]:
%%timeit -n1
model.fit(X_train, y_train)

1 loop, best of 5: 46.8 ms per loop


In [118]:
%%timeit -n1
y_predict = model.predict(X_test)

1 loop, best of 5: 10.9 ms per loop


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)
model.fit(X_train, y_train)

print(" " * 20 + "Without Scaler:")
print(classification_report(y_test, model.predict(X_test)))

                    Without Scaler:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89     27882
           1       0.62      0.56      0.59      7667

    accuracy                           0.83     35549
   macro avg       0.75      0.73      0.74     35549
weighted avg       0.83      0.83      0.83     35549



In [120]:
X_train, X_test, y_train, y_test = train_test_split(X_mm, y, test_size=0.25, shuffle=False)
model.fit(X_train, y_train)

print(" " * 20 + "MinMaxScaler:")
print(classification_report(y_test, model.predict(X_test)))

                    MinMaxScaler:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89     27882
           1       0.62      0.56      0.59      7667

    accuracy                           0.83     35549
   macro avg       0.75      0.73      0.74     35549
weighted avg       0.83      0.83      0.83     35549



In [121]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.25, shuffle=False)
model.fit(X_train, y_train)

print(" " * 20 + "StandartScaler:")
print(classification_report(y_test, model.predict(X_test)))

                    StandartScaler:
              precision    recall  f1-score   support

           0       0.88      0.91      0.89     27882
           1       0.62      0.56      0.59      7667

    accuracy                           0.83     35549
   macro avg       0.75      0.73      0.74     35549
weighted avg       0.83      0.83      0.83     35549



Вывод

Метрики разных моделей:
1. Логистическая регрессия: f1 = 0.91 для 0 и f1 = 0.58 для 1 
2. Метод ближайших соседей: 0.91 и 0.54 (без масштабирования данных и n_neighbors = 15)
3. Байесовский классификатор: 0.89 и 0.59

В данной задаче лучше всего себя показала логистическая регрессия. У этой  
модели довольно высокие показатели precision (0.87 и 0.75) и хорошие показатели recall (0.96 и 0.47). Метрики других моделей немного хуже. При этом метод ближайших соседей работает на порядок дольше логистической регресси, и на несколько порядков дольше байесовского классификатора, поэтому я думаю что данный метод не подходит для данной задачи. Байесовский классификатор имеет самую низкую точность для класса 1, но время его работы значительно ниже чем у других алгоритмов, поэтому в задачах где важна скорость выполнения целесообразнее использовать данный метод машинного обучения.


### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [122]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, -1, -1])

In [123]:
import math

def probability(theta, X):
    result = np.zeros(X.shape[0])
    for i in range(result.size):
        result[i] = 1 / (1 + math.exp(-np.dot(X[i], theta)))
    return result
prob = probability(theta, X)


assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [128]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    result = np.zeros_like(prob)
    result[prob.round(3) <= 0] = -1
    result[prob.round(3) > 0] = 1
    return result

y_pred = binary_class_prediction(theta, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == -1, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$ 

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$ 

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell} 
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$ 
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [125]:
def logloss(theta, X, y): 
    result = 0
    for i in range(y.size):
        result += np.log(1 + math.exp(- y[i] * np.dot(theta, X[i])))
    result /= y.size
    return result

In [126]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>: 

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [127]:
print()
def gradient(theta, X, y):
    h = 1
    n = np.random.randint(1, X.shape[0])
    result = theta.copy()
    for i in range(y.size):
        result += math.exp(- y[i] * np.dot(theta, X[i])) * y[i] * X[i] / (1 + math.exp(- y[i] * np.dot(theta, X[i])))
    result /= y.size
    return result 

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

Функция обучения уже реализована

In [ ]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [ ]:
X, y = make_classification(n_samples=2000)

In [ ]:
optimal_theta = fit(X, y)

In [ ]:
y_pred = binary_class_prediction(optimal_theta, X)